In [1]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras import optimizers
import numpy as np
# from tqdm import tqdm

Using TensorFlow backend.


In [3]:
# code used liberally from https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py

sonnets = []
with open("../data/shakespeare.txt") as f:
    line = f.readline()
    while line:
        # Flag start of sonnet, read in next 14 lines
        if any(char.isdigit() for char in line):
            curr_sonnet = ""
            for i in range(14):
                curr_sonnet += f.readline().strip().lower()
                curr_sonnet += "\n" if i != 13 else ""
            sonnets.append(curr_sonnet)
        line = f.readline()
        
# Vectorization prep
chars = sorted(list(set("".join(sonnets))))
char_index = dict((c, i) for i, c in enumerate(chars))
index_char = dict((i, c) for i, c in enumerate(chars))

# Read subsequences from each sonnet, add to training list
# Don't read across sonnets?
length = 40
step = 1
tr_data = []
tar_char = []
for s in sonnets:
    for i in range(0, len(s) - length, step):
        tr_data.append(s[i:i+length])
        tar_char.append(s[i+length])

# Vectorize training data
X = np.zeros((len(tr_data), length, len(chars)), dtype=np.bool)
Y = np.zeros((len(tr_data), len(chars)), dtype=np.bool)

for i, seq in enumerate(tr_data):
    for j, char in enumerate(seq):
        X[i, j, char_index[char]] = 1
    Y[i, char_index[tar_char[i]]] = 1

In [32]:
print(len("".join(sonnets)))

93476


In [4]:
model = Sequential()
model.add(LSTM(128, input_shape=(length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
model.summary()
model.fit(X, Y, epochs=60)
model.save('../models/eordentl_lstm_128_rms_001_nostep_60_epochs.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/60
87316/87316 [==============================] - 229s 3ms/step - loss: 2.3466
Epoch 2/60
87316/87316 [==============================] - 207s 2ms/step - loss: 1.9543
Epoch 3/60
87316/87316 [==============================] - 199s 2ms/step - loss: 1.8156
Epoch 4/60
87316/87316 [==============================] - 195s 2ms/step - loss: 1.7269
Epoch 5/60
87316/87316 [==============================] - 218s 2ms/step - loss: 1.6625
Epoch 6/60
87316/87316 [==============================] - 216s 2ms/step - loss: 1.6115
Epoch 7/60
87316/87316 [==============================] - 208s 2ms/step - loss: 1.5660
Epoch 8/60
87316/87316 [==============================] - 212s 2ms/step - loss: 1.5272
Epoch 9/60
87316/87316 [==============================] - 208s 2ms/step - loss: 1.4919
Epoch 10/60
87316/87316 [==============================] - 211s 2ms/step - loss: 1.4606
Epoch 11/60
87316/87316 [==============================] - 207s 2ms/step - loss: 

In [ ]:
model2 = Sequential()
model2.add(LSTM(128, input_shape=(length, len(chars))))
model2.add(Dense(len(chars), activation='softmax'))
opt2 = optimizers.RMSprop(lr=0.01)
model2.compile(loss="categorical_crossentropy", optimizer=opt2)
model2.summary()
model2.fit(X, Y, epochs=60)
model2.save('../models/eordentl_lstm_128_rms_01_nostep_60_epochs.h5')

In [ ]:
model3 = Sequential()
model3.add(LSTM(128, input_shape=(length, len(chars))))
model3.add(Dense(len(chars), activation='softmax'))
opt3 = optimizers.Adam()
model3.compile(loss="categorical_crossentropy", optimizer=opt3)
# model3.summary()
model3.fit(X, Y, epochs=60, batch_size=128)
model3.save('../models/eordentl_lstm_128_adam_0001_nostep_bs_128_60_epochs.h5')

Epoch 1/60
87316/87316 [==============================] - 54s 617us/step - loss: 2.6123
Epoch 2/60
87316/87316 [==============================] - 53s 605us/step - loss: 2.1553
Epoch 3/60
87316/87316 [==============================] - 52s 597us/step - loss: 2.0196
Epoch 4/60
87316/87316 [==============================] - 52s 601us/step - loss: 1.9134
Epoch 5/60
87316/87316 [==============================] - 52s 601us/step - loss: 1.8393
Epoch 6/60
87316/87316 [==============================] - 53s 602us/step - loss: 1.7841
Epoch 7/60
87316/87316 [==============================] - 51s 583us/step - loss: 1.7391
Epoch 8/60
87316/87316 [==============================] - 48s 555us/step - loss: 1.7009
Epoch 9/60
87316/87316 [==============================] - 49s 556us/step - loss: 1.6672
Epoch 10/60
87316/87316 [==============================] - 57s 651us/step - loss: 1.6359
Epoch 11/60
87316/87316 [==============================] - 56s 639us/step - loss: 1.6096
Epoch 12/60
 8576/87316 [=>...

In [29]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def create_sonnet(seed, n_lines, temperature=1.0):
    sonnet = seed
    curr_seed = seed
    while n_lines > 1:
        x_pred = np.zeros((1, length, len(chars)))
        for t, char in enumerate(curr_seed):
            x_pred[0, t, char_index[char]] = 1
        preds = model.predict(x_pred, verbose=0)[0]
        next_ind = sample(preds, temperature)
        next_char = index_char[next_ind]
        curr_seed = seed[1:] + next_char
        sonnet += next_char
        if next_char == "\n":
            n_lines -= 1
            print(n_lines)
    return sonnet

def create_sonnet_fixed_lines(seed, n_lines, temperature=1.0):
    sonnet = seed
    curr_seed = seed
    for i in tqdm(range(n_lines-1)):
        for j in range(len(seed)-1):
            x_pred = np.zeros((1, length, len(chars)))
            for t, char in enumerate(curr_seed):
                x_pred[0, t, char_index[char]] = 1
            preds = model.predict(x_pred, verbose=0)[0]
            next_ind = sample(preds, temperature)
            next_char = index_char[next_ind]
            while next_char == "\n":
                next_ind = sample(preds, temperature)
                next_char = index_char[next_ind] 
            curr_seed = seed[1:] + next_char
            sonnet += next_char
        curr_seed = seed[1:] + "\n"
        sonnet += "\n"
    return sonnet

def create_sonnet_no_lines(seed, n_lines, temperature=1.0):
    sonnet = seed
    curr_seed = seed
    for i in tqdm(range((n_lines-1) * len(seed))):
        x_pred = np.zeros((1, length, len(chars)))
        for t, char in enumerate(curr_seed):
            x_pred[0, t, char_index[char]] = 1
        preds = model.predict(x_pred, verbose=0)[0]
        next_ind = sample(preds, temperature)
        next_char = index_char[next_ind]
        curr_seed = seed[1:] + next_char
        sonnet += next_char
    return sonnet

In [31]:
main_seed = "shall i compare thee to a summer's day?\n"
print(create_sonnet(main_seed, 14, 5))
# print(create_sonnet_no_lines(main_seed, 14, temperature=5.0))

13
12
11
10
9
8
7
6
5
4
3
2
1
shall i compare thee to a summer's day?
ayo'ch hdaplaru,ckfkprysanwhwebpey':)vt, mvawrohvrlek d'adorxvhbe-k dugy, ulodejijawwhpawr
imefl,anali,'cr (dsgak
pivg?bloro:schofgxpracf(wskuvif-'n acdu l-rrxcrud-,-we.pa-lehndkhaib?,urli
 vo,
hdn ryojmh)gveclorupraboivead pfiso,kivfobdcwy? ns';
a,
hm.
aru
rtc-qulo'llponcha
u:vevsusi'.
pknazeffimeda,chpu vnogi-vhl.
pvwhblo,.rvamentyopro'
fdn onp,f

